In [30]:
import os

In [31]:
%pwd

'c:\\Users\\Desktop\\xReayDetection'

In [14]:
os.chdir('../')

In [15]:
%pwd

'c:\\Users\\Desktop\\xReayDetection'

In [32]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [33]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [34]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/SRVHCCC/kedney-desease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [35]:
import tensorflow as tf
from pathlib import Path
import dagshub
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import json

In [38]:
import json

In [39]:
class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Ensure the model and valid_generator are available
        if self.model and self.valid_generator:
            self.score = self.model.evaluate(self.valid_generator)
            self.save_score()
        else:
            raise ValueError("Model or validation generator could not be loaded properly.")

    def save_score(self):
        # Ensure score has valid values
        if self.score and len(self.score) >= 2:
            scores = {"loss": self.score[0], "accuracy": self.score[1]}
            self.save_json(path=Path("scores.json"), data=scores)
        else:
            raise ValueError("Score should have at least loss and accuracy.")

    @staticmethod
    def save_json(path: Path, data: dict):
        # Saves the data into a JSON file
        with open(path, "w") as json_file:
            json.dump(data, json_file)

    def log_into_mlflow(self):
        try:
            # Set the tracking URI and validate
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Start an MLflow run
            with mlflow.start_run():
                # Log parameters
                if isinstance(self.config.all_params, dict):
                    mlflow.log_params(self.config.all_params)
                else:
                    raise ValueError("self.config.all_params must be a dictionary")

                # Log metrics (ensure score has loss and accuracy)
                if len(self.score) >= 2:
                    mlflow.log_metrics(
                        {"loss": self.score[0], "accuracy": self.score[1]}
                    )
                else:
                    raise ValueError("self.score should have at least 2 values: [loss, accuracy]")

                # Log and register the model
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
        
        except Exception as e:
            print(f"Error logging into MLflow: {e}")
            raise

In [40]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Directory created or already exists: artifacts
Found 3732 images belonging to 4 classes.
234/234 [==============================] - 1499s 6s/step - loss: 21.4525 - accuracy: 0.2219


2024/09/20 16:54:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/09/20 16:55:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2024/09/20 16:56:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
2024/09/20 16:56:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run dapper-ox-611 at: https://dagshub.com/SRVHCCC/kedney-desease-classification.mlflow/#/experiments/0/runs/5d5044da7fee4a748af56b070a6403d5.
2024/09/20 16:56:51 INFO mlflow.tracking._tracking_service.client: 🧪 